In [ ]:
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

#Read Data

In [ ]:
def dropConfidence(df):
    drop_idx = list(range(2, df.shape[1], 3))
    df = df.drop(drop_idx, axis=1)
    return df

def getXcoords(df):
    drop_idx = list(range(1, df.shape[1], 2))
    df = df.drop(drop_idx, axis=1)
    return df

def getYcoords(df):
    drop_idx = list(range(0, df.shape[1], 2))
    df = df.drop(drop_idx, axis=1)
    return df


In [ ]:
def getData():
    with open("keypoints5.json") as jsonFile:
        jsonObject1 = json.load(jsonFile)
        jsonFile.close()
        #print((len(jsonObject1)))
        df = pd.DataFrame.from_records(jsonObject1)
        df = dropConfidence(df)
        df.columns = np.arange(len(df.columns))
        #print(df)
        x = getXcoords(df)
        #print(x.loc[0])
        y = getYcoords(df)
        print(df.shape[0])
        for i in range(df.shape[0]):
            #print("i:", i)
            fig, ax = plt.subplots()
            #fig.set_size_inches(18.5, 10.5)
            x_vals = x.loc[i]
            y_vals = y.loc[i]
            ax.plot(x_vals, y_vals, 'x')
            fail_list= []
            for j in range(25):
              if x_vals.iloc[j] == 0:
                fail_list.append(j)
              plt.annotate(str(j), (x_vals.iloc[j], y_vals.iloc[j]))
            print(fail_list)
            ax.set_ylim(ax.get_ylim()[::-1])
            #ax.set_xbound(lower=1000, upper=2000)
            #plt.title('%s Plot' %i)
            #plt.savefig('plot/%s Plot' %i)
            #ax[1].invert_yaxis()
            #plt.show()
            plt.close(fig)
    return df            

In [ ]:
df = getData()


In [ ]:
!zip -r /plot.zip /content/plot

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
data = np.ndarray([])

In [ ]:
s = pd.Series(data, index=index)

In [ ]:
def getLabel():
  my_file = open("test5", "r")
  content_list = my_file.read().splitlines()
  #content_list = list(map(int, content_list))
  print(type(int(content_list[(0)])))
  contentlisti = []
  for i in range(len(content_list)):
    contentlisti.append(int(content_list[i]))

  return contentlisti  

#labels = getLabel()

In [ ]:
labels = np.zeros(2253)
a = range(0, 500, 1)
b = range(500, 1000, 1)
c = range(1000, 1500, 1)
d = range(1000, 2253, 1)
labels[a] = 1
labels[b] = 2
labels[c] = 3
labels[d] = 4
print(labels)

[1. 1. 1. ... 4. 4. 4.]


#LSTM

In [ ]:
frames = getData()
timeseries = frames.to_numpy()

In [ ]:
def temporalize(X, y, lookback):
    output_X = []
    output_y = []
    for i in range(len(X)-lookback-1):
        t = []
        for j in range(1,lookback+1):
            # Gather past records upto the lookback period
            t.append(X[[(i+j+1)], :])
        output_X.append(t)
        output_y.append(y[i+lookback+1])
    return output_X, output_y

#need Valset

In [ ]:
timesteps = 1
n_features = 4

labels = np.asarray(labels)
print(timeseries.shape)
X, x = temporalize(X = timeseries, y = labels, lookback = timesteps)
X = np.array(X)
print(X.shape)
X = X.reshape(X.shape[0], timesteps, n_features)
from keras.utils.np_utils import to_categorical
x = to_categorical(x)
print(x.shape)

X_test = X[-10:]
y_test = x[-10:]
print(x.shape)
print(X.shape)

In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.LSTM(32, return_sequences=True, stateful=False, input_shape = (timesteps,50)))
model.add(keras.layers.LSTM(20))
model.add(keras.layers.Dense(5, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=["accuracy"])
model.summary()
print("Train...")
model.fit(X, x, batch_size=8, epochs=500, validation_data=(X_test, y_test))

In [ ]:
filepath = './saved_model'
save_model(model, filepath)

In [ ]:
model = Sequential()
model.add(LSTM(128, activation='relu', input_shape=( timesteps,n_features), return_sequences=True))
model.add(LSTM(50, activation='relu'))
model.add(Dense(5, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy')
model.summary()

# fit model
model.fit(X, x, epochs=300, batch_size=5, verbose=1,  validation_data=(X_test, y_test))